# Advanced Sequential Logic Exercise
Click the ▷ button below to setup the exercise's environment (will take about 2 mins).


In [ ]:
#@title Install dependencies {display-mode: "form"}
#@markdown - Click the ▷ button to setup the digital design environment

import os
import pathlib
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     --channel conda-forge \
                     iverilog \
                     verilator
!python -m pip install numpy
PATH = os.environ['PATH']
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}


In [ ]:
#@title Click the ▷ button to setup testbench
#@markdown The testbench allows us to check your work. No need to pay attention to this for now.

%%writefile tb_stack.v

`timescale 1ns/1ps
module tb_stack;
	reg clk, nrst, push, pop;
	reg [3:0] din;
	wire [3:0] dout;

	stack uut (
		.clk(clk),
		.nrst(nrst),
		.push(push),
		.pop(pop),
		.din(din),
		.dout(dout)
	);

	// Clock generation
	initial clk = 0;
	always #5 clk = ~clk;

    reg [3:0] push_values [0:3];
    integer err;

    // Waveform dump
    initial begin
        $dumpfile("tb_stack.vcd");
        $dumpvars(0, tb_stack);
    end

	initial begin
		$display("Starting stack testbench...");
		nrst = 0; push = 0; pop = 0; din = 0;
		#12;
		nrst = 1;
		#10;

        push_values[0] = 4'd3;
        push_values[1] = 4'd7;
        push_values[2] = 4'd12;
        push_values[3] = 4'd15;

		// Push 4 values
		for (integer i = 0; i < 4; i = i + 1) begin
			@(negedge clk);
			push = 1;
            pop = 0;
            din = push_values[i];
			$display("Pushing %d", din);
			@(negedge clk);
			push = 0;
			#2;
		end

        #10;

		// Pop 4 values
		for (integer i = 0; i < 4; i = i + 1) begin
			@(negedge clk);
			push = 0;
            pop = 1;
			@(negedge clk);
			pop = 0;
			#2;
            if (dout !== 4'bxxxx)
                if (dout == push_values[3 - i])
                    $display("Popped value: %d (correct)", dout);
                else
                    $display("Popped value: %d (incorrect, expected %d)", dout, push_values[3 - i]);
            else
                $display("Popped value: Invalid (stack underflow)");
		end

		#10;
		$finish;
	end
endmodule




# The exercise

- A stack is a data structure where the first element "pushed" is the last one "popped", like an actual stack of items.
- For example, enqueueing 3, 4, 6, 7 in order and then dequeueing all of them should yield 7, 6, 4, 3 in order.
- When push is 1, the 4b input in **din** is pushed to the stack upon the next clock edge.
- When pop is 1, the 4b output **dout** will be popped from the stack upon the next clock edge.
- The stack can only contain at most 4 elements. Trying to push more than 4 elements will not work, and so will pop not work when the stack is empty.
- For example, pushing 3, 4, 6, 7, 8, and then popping should yield 7, 6, 4, 3, 0, since 8 was not pushed into the stack.
- Your task is to fill in the behavioral model of the stack module below.

A sample waveform is shown below:

![](https://lawrence-lugs.notion.site/image/attachment%3A139a5a17-281d-402a-bb60-049712d9f2c4%3Awavedrom.png?table=block&id=268ab8b4-50a9-80d7-a8c0-e5b605d21f96&spaceId=378d1135-5228-4759-9c1d-b2c3e4e47feb&width=1420&userId=&cache=v2)

Fill in your verilog code below:

In [ ]:
%%writefile stack.v

module stack (
    input wire clk,
    input wire nrst,
    input wire push,
    input wire pop,
    input wire [3:0] din,
    output reg [3:0] dout
);

    // Write your stack implementation here

endmodule

# Checking

Press the ▷ button to check your work

In [ ]:
%cd content
! iverilog tb_stack.v stack.v
! ./a.out